In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/log-anamoly1/anomaly_label.csv
/kaggle/input/logfile/cleaned_data.csv


In [11]:
import tensorflow as tf
import numpy as np
from keras.layers import Embedding, Conv1D, MaxPooling1D
from keras.layers import Flatten, Dropout, Dense
import torch
from keras.models import Sequential
from torch.autograd import Variable
from torch import IntTensor
from sklearn.model_selection import train_test_split

import re
import string
import emoji
import itertools
import collections
import pandas as pd
from nltk import word_tokenize, pos_tag
import warnings
from collections import defaultdict, Counter
from sklearn.preprocessing import LabelEncoder
warnings.filterwarnings("ignore")
from sklearn.metrics import classification_report
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.metrics import precision_recall_fscore_support
from nltk.probability import FreqDist

In [3]:
import pandas as pd
data = pd.read_csv("../input/logfile/cleaned_data.csv")
data['X_data'] = data['X_data'].apply(lambda s: s.split(' ', 1)[1])
data

,X_data,Y_data
0,81109 203615 148 INFO dfs DataNode PacketRespo...,0
1,81109 203807 222 INFO dfs DataNode PacketRespo...,0
2,81109 204005 35 INFO dfs FSNamesystem BLOCK Na...,0
3,81109 204015 308 INFO dfs DataNode PacketRespo...,0
4,81109 204106 329 INFO dfs DataNode PacketRespo...,0
...,...,...
1995,81111 101621 24902 INFO dfs DataNode DataXceiv...,0
1996,81111 101735 26595 INFO dfs DataNode PacketRes...,0
1997,81111 101804 26494 INFO dfs DataNode DataXceiv...,0
1998,81111 101954 26414 INFO dfs DataNode PacketRes...,0


In [4]:
X = list(data['X_data'])

In [5]:
words_in_train = [sent.split() for sent in X] # Splitting the tweet into tokens
all_words_train = list(itertools.chain(*words_in_train)) # Making a flatlist of all the words in the corpus
all_words = all_words_train  
corpus = X  
vocab_size = len(set(all_words)) # number of words to keep.
embedding_dim = 100 # Dimension of the vector representation for each word 
max_length = 0
for i in range(len(corpus)):
    x = word_tokenize(corpus[i])
    if len(x) > max_length:
        max_length = len(x)

In [6]:
print("Max length of the sequence :",max_length)
print("Size of the vocabulary :",vocab_size)

Max length of the sequence : 225
Size of the vocabulary : 5972


In [7]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token="<OOV>")
tokenizer.fit_on_texts(data['X_data'])
word_index = tokenizer.word_index

In [8]:
# Pad the sequences so that they are all the same length
training_sequences = tokenizer.texts_to_sequences(data['X_data'])
trainds_vec = tf.keras.preprocessing.sequence.pad_sequences(training_sequences,maxlen=max_length, padding='post')

In [9]:
# import library
from imblearn.over_sampling import SMOTE
from collections import Counter

smote = SMOTE()

# fit predictor and target variable
x_smote, y_smote = smote.fit_resample(np.array(trainds_vec), data['Y_data'])
print('Original dataset shape', np.array(trainds_vec).shape)
print('Resample dataset shape', x_smote.shape)


Original dataset shape (2000, 225)
Resample dataset shape (3862, 225)


In [12]:
X_train,X_test,Y_train,Y_test = train_test_split(x_smote,y_smote,test_size=0.2)

In [13]:
Training_labels = tf.keras.utils.to_categorical(Y_train)
Testing_labels = tf.keras.utils.to_categorical(Y_test)

In [14]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
]

2022-10-31 10:20:39.750669: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:20:39.751741: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:20:39.752756: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:20:39.753557: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-31 10:20:39.754321: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from S

In [24]:
filter_size = 64
kernel_size = 5
pool_size = 5


model = tf.keras.Sequential([
#    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length,
#                              embeddings_initializer='orthogonal', embeddings_regularizer='L2'),
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Conv1D(filter_size, kernel_size, activation='relu', padding='same'),
    tf.keras.layers.MaxPooling1D(pool_size, padding='same'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(100)),
    tf.keras.layers.Dense(32, activation=tf.nn.relu),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

model.compile(
    loss=tf.keras.losses.CategoricalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=METRICS
)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 225, 100)          597200    
_________________________________________________________________
conv1d_3 (Conv1D)            (None, 225, 64)           32064     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, 45, 64)            0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 45, 64)            0         
_________________________________________________________________
bidirectional_3 (Bidirection (None, 200)               132000    
_________________________________________________________________
dense_7 (Dense)              (None, 32)                6432      
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)               

In [25]:
# callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', patience=10, mode='max', restore_best_weights=True)
tf.config.run_functions_eagerly(True)

num_epochs = 5
history = model.fit(
    X_train,
    Training_labels,
    epochs=num_epochs,
    batch_size = 128,
    validation_data=(X_test, Testing_labels)
)

Epoch 1/5
25/25 [==============================] - 2s 63ms/step - loss: 0.6384 - accuracy: 0.6414 - precision: 0.6414 - recall: 0.6414 - auc: 0.6904 - val_loss: 0.6469 - val_accuracy: 0.6869 - val_precision: 0.6869 - val_recall: 0.6869 - val_auc: 0.7792
Epoch 2/5
25/25 [==============================] - 2s 61ms/step - loss: 0.4159 - accuracy: 0.8174 - precision: 0.8174 - recall: 0.8174 - auc: 0.8953 - val_loss: 0.6858 - val_accuracy: 0.7400 - val_precision: 0.7400 - val_recall: 0.7400 - val_auc: 0.8065
Epoch 3/5
25/25 [==============================] - 2s 62ms/step - loss: 0.1176 - accuracy: 0.9550 - precision: 0.9550 - recall: 0.9550 - auc: 0.9914 - val_loss: 0.7914 - val_accuracy: 0.7503 - val_precision: 0.7503 - val_recall: 0.7503 - val_auc: 0.8333
Epoch 4/5
25/25 [==============================] - 2s 66ms/step - loss: 0.0442 - accuracy: 0.9851 - precision: 0.9851 - recall: 0.9851 - auc: 0.9978 - val_loss: 1.6134 - val_accuracy: 0.6856 - val_precision: 0.6856 - val_recall: 0.6856 - 